In [30]:
import numpy as np
import random

#define the transition (default size = 5x5)
def next_state_reward(state, action, height = 5, width = 5):
  action_move = [(-1, 0), (1, 0), (0, -1), (0, 1)] #4 possible actions -  0: Up / 1: Down / 2: Left / 3: Right

  #state transition: states = (height, width)
  state[0] += action_move[action][0]
  state[1] += action_move[action][1]

  #treating some blocked moves
  if state[0] < 0:
    state[0] = 0
  elif state[0] >  height-1:
    state[0] = height-1
  if state[1] < 0:
    state[1] = 0
  elif state[1] > width-1:
    state[1] = width-1

  #return the new state according to the selected action and reward = -1
  return [state[0], state[1]], -1


In [31]:
def mysterious_state_reward(state, action, height = 5, width = 5):
  action_move = [(-1, 0), (1, 0), (0, -1), (0, 1)]

  mys = []

  mys_1 = [1,3]
  mys_2 = [0,3]

  if state in ([0,2] or [2,1]):
    mys = mys_1
  else:
    mys = mys_2

  a = random.choice(mys)
  state[0] += action_move[a][0]
  state[1] += action_move[a][1]

  if state[0] < 0:
    state[0] = 0
  elif state[0] >  height-1:
    state[0] = height-1
  if state[1] < 0:
    state[1] = 0
  elif state[1] > width-1:
    state[1] = width-1

  return [state[0], state[1]], 2

In [32]:
def policy_evaluation(action, policy, height = 5, width = 5, theta=1e-5, gamma=0.9):
  #initialize state value function
  value_table = np.zeros(shape=(height, width))

  #two-array version (try in-place version by yourself)
  iter = 0
  while iter<2000:
    #update the new state value function
    next_value_table = np.zeros(shape=(height, width))
    delta = 0
    for i in range(height):
      for j in range(width):

        #always assign 0 to the terminal states
        if ((i == 0) and (j == 0)) or ((i == height-1) and (j == height-1)):
          value_iter = 0

        else:
          #update the non-terminal states using the Bellman equation
          value_iter = 0

          for act in action:
            if [i,j] in [[0,2],[2,1],[3,3]]:
                transition_prob = 0.5
                next_s, r = mysterious_state_reward([i,j], act)

            else:
                transition_prob = 1 #deterministic
                next_s, r = next_state_reward([i,j], act)
            value_iter += policy[i][j][act]*transition_prob*(r + gamma*value_table[next_s[0]][next_s[1]])

          #deterministic state transition (do not consider the transition probability in this case)
          # for act in action:
          #   next_s, r = next_state_reward([i,j], act)
          #   value_iter += policy[i][j][act]*transition_prob*(r + gamma*value_table[next_s[0]][next_s[1]])

        #update the state value function
        next_value_table[i][j] = round(value_iter, 3)

        #compare the error and the error bound
        delta = max(delta, abs(next_value_table[i][j] - value_table[i][j]))

    value_table = next_value_table
    iter += 1

    #termination condition
    if delta < theta:
      # print('Final results ({} iterations): \n {}'.format(iter, next_value_table))
      break

    #print the results
    # iter_visual = [1, 2, 10, 50] + [n*100 for n in range(20)]
    # if iter in iter_visual:
    #   print('iteration {}: \n {}'.format(iter, next_value_table))

  return next_value_table

policy_evaluation in-place update

In [33]:
def policy_evaluation_in_place(action, policy, height=5, width=5, theta=1e-5, gamma=0.9):
    # Initialize state value function
    value_table = np.zeros(shape=(height, width))

    iter = 0

    while True:
        delta = 0

        for i in range(height):
            for j in range(width):

                # Always assign 0 to the terminal states
                if ((i == 0) and (j == 0)) or ((i == height-1) and (j == width-1)):
                    continue

                old_value = value_table[i][j]

                # Update non-terminal states using the Bellman equation
                value_iter = 0
                for act in action:
                    if [i,j] in [[0,2],[2,1],[3,3]]:
                      transition_prob = 0.5
                      next_s, r = mysterious_state_reward([i,j], act)

                    else:
                      transition_prob = 1 #deterministic
                      next_s, r = next_state_reward([i,j], act)
                    value_iter += policy[i][j][act]*transition_prob*(r + gamma*value_table[next_s[0]][next_s[1]])

                value_table[i][j] = round(value_iter, 3)

                # Compare the error and update delta
                delta = max(delta, abs(old_value - value_table[i][j]))

        iter += 1

        if delta < theta:
            break

        # iter_visual = [1, 2, 10, 50] + [n*100 for n in range(20)]
        # if iter in iter_visual:
        #     print('iteration {}: \n {}'.format(iter, value_table))

    return value_table


In [34]:
#grid environment
grid_height, grid_width = 5, 5 #5x5 gridworld
action = [0, 1, 2, 3] #0: up / 1: down / 2: left / 3: right

#policy initialization
policy = np.zeros(shape=(grid_height, grid_width, len(action)))

#random equiprobable policy
for i in range(grid_height):
  for j in range(grid_width):
    for k in range(len(action)):
      if ((i == 0) and (j == 0)) or ((i == grid_height-1) and (j == grid_height-1)):
        policy[i][j][k] = 0
      else:
        policy[i][j][k] = 0.25


state_value = policy_evaluation(action, policy)
# in_place_state_value = policy_evaluation_in_place(action, policy)
print('Vㅠ(s):',state_value)
# print('V(s):',in_place_state_value)

Vㅠ(s): [[ 0.    -2.182  0.028 -4.592 -6.297]
 [-3.48  -3.355 -3.762 -5.334 -6.198]
 [-4.212 -0.729 -4.    -4.404 -5.239]
 [-5.86  -4.828 -4.276 -0.338 -2.97 ]
 [-6.725 -6.152 -5.18  -2.955  0.   ]]


In [35]:
def policy_improvement(value, action, policy, height = 5, width = 5, gamma = 0.9):
  #improved policy
  new_policy = np.zeros(shape=(height, width, len(action)))

  for i in range(height):
    for j in range(width):
      if ((i == 0) and (j == 0)) or ((i == height-1) and (j == height-1)):
        pass

      else:
        #compute q(s, a) for every state
        q_list = []

        for k in action:
          if [i,j] in [[0,2],[2,1],[3,3]]:
              transition_prob = 0.5
              next_s, r = mysterious_state_reward([i,j], k)

          else:
              transition_prob = 1 #deterministic
              next_s, r = next_state_reward([i,j], k)
          q_list.append(transition_prob*(r + gamma*value[next_s[0]][next_s[1]]))

        #find the argmax
        max_actions = [act for act, x in enumerate(q_list) if x == max(q_list)]

        #update the policy (new equiprobable)
        for k in action:
          if k in max_actions:
            new_policy[i][j][k] = 1/len(max_actions)

  print('Updated policy is {}'.format(new_policy))
  return new_policy

In [36]:
improved_policy = policy_improvement(state_value, action, policy)
print(improved_policy)

Updated policy is [[[0.         0.         0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.5        0.         0.5        0.        ]
  [0.         0.         1.         0.        ]
  [0.         0.         1.         0.        ]]

 [[1.         0.         0.         0.        ]
  [0.         1.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [0.         0.         1.         0.        ]
  [0.         1.         0.         0.        ]]

 [[0.         0.         0.         1.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.         0.         1.         0.        ]
  [0.         1.         0.         0.        ]
  [0.         1.         0.         0.        ]]

 [[1.         0.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.33333333 0.         0.33333333 0.33333333]
  [0.         1.         0.         0.        ]]

 [[1.         

In [39]:
def policy_iteration(action, init_policy, height = 5, width = 5):
  #initial policy
  policy = init_policy

  while True:
    #false if updated policy is not equal to original policy
    policy_stable = True

    #policy evaluation & improveemnt
    state_value = policy_evaluation(action, policy)
    # in_place_state_value = policy_evaluation_in_place(action, policy)
    improved_policy = policy_improvement(state_value, action, policy)
    # improved_policy_in_place = policy_improvement(in_place_state_value, action, policy)

    #check whether the two policies are identical
    for i in range(height):
      for j in range(width):
        for k in range(len(action)):
         if policy[i][j][k] != improved_policy[i][j][k]:
          policy_stable = False

    #update the policy
    policy = improved_policy
    # policy = improved_policy_in_place

    #termination condition
    if policy_stable:
      break

  return policy


In [40]:
print(policy_iteration(action, policy))

Updated policy is [[[0.         0.         0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.         0.         0.         1.        ]
  [0.         0.         1.         0.        ]
  [0.         0.         1.         0.        ]]

 [[1.         0.         0.         0.        ]
  [0.         1.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [0.         0.         1.         0.        ]
  [0.         1.         0.         0.        ]]

 [[0.         0.         0.         1.        ]
  [0.33333333 0.         0.33333333 0.33333333]
  [0.         0.         1.         0.        ]
  [0.         1.         0.         0.        ]
  [0.         1.         0.         0.        ]]

 [[1.         0.         0.         0.        ]
  [1.         0.         0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.5        0.         0.         0.5       ]
  [0.         1.         0.         0.        ]]

 [[1.         